<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/mnist/Colab_Notebook_MNIST_Do_Label_Shift_Adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download predictions from zenodo

In [0]:
!wget https://zenodo.org/record/3437666/files/test_labels.txt.gz?download=1 -O test_labels.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-0.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-0.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-10.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-10.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-20.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-20.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-30.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-30.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-40.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-40.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-50.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-50.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-60.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-60.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-70.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-70.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-80.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-80.txt.gz
!wget https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-90.txt.gz?download=1 -O testpreacts_model_mnist_set-16000_seed-90.txt.gz
!wget https://zenodo.org/record/3437666/files/valid_labels.txt.gz?download=1 -O valid_labels.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-0.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-0.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-10.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-10.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-20.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-20.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-30.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-30.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-40.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-40.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-50.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-50.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-60.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-60.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-70.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-70.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-80.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-80.txt.gz
!wget https://zenodo.org/record/3437666/files/validpreacts_model_mnist_set-16000_seed-90.txt.gz?download=1 -O validpreacts_model_mnist_set-16000_seed-90.txt.gz

--2019-09-22 19:16:08--  https://zenodo.org/record/3437666/files/test_labels.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13746 (13K) [application/octet-stream]
Saving to: ‘test_labels.txt.gz’

test_labels.txt.gz  100%[===================>]  13.42K  --.-KB/s    in 0.1s    

2019-09-22 19:16:10 (114 KB/s) - ‘test_labels.txt.gz’ saved [13746/13746]

--2019-09-22 19:16:10--  https://zenodo.org/record/3437666/files/testpreacts_model_mnist_set-16000_seed-0.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 476854 (466K) [application/octet-stream]
Saving to: ‘testpreacts_model_mnist_set-16000_seed-0.txt.gz’

testpreacts_model_m 100%[===================>] 465.68K   791KB/s    in 0.6s    

2019-09-22 19:1

In [0]:
!gunzip -f *.gz
!md5sum *.txt

b7cac25c27a1c23e8afea9173a7dc13e  test_labels.txt
8b3d5dc6b17c514a68f8cffe6415d278  testpreacts_model_mnist_set-16000_seed-0.txt
5b749df4cd3ea08a43789dabf05becc5  testpreacts_model_mnist_set-16000_seed-10.txt
b4225781e18cd714e5b4155f345e8491  testpreacts_model_mnist_set-16000_seed-20.txt
bcbde2b0701ca48bc33c8a5df95c5ce7  testpreacts_model_mnist_set-16000_seed-30.txt
74f547450bf2a477d4c54452c81f97e5  testpreacts_model_mnist_set-16000_seed-40.txt
c1511b9fe6dc30b1c13576e7c22ca658  testpreacts_model_mnist_set-16000_seed-50.txt
42fa4f1ce2e71ec3e2ec791baed3e833  testpreacts_model_mnist_set-16000_seed-60.txt
c49b0bb9553cec0d3ceb0eee36f01705  testpreacts_model_mnist_set-16000_seed-70.txt
63fb5b4fb6a0c45e5b25bbf296d86b1f  testpreacts_model_mnist_set-16000_seed-80.txt
08500b34a2bf3a2f7ea58d6e75825fdd  testpreacts_model_mnist_set-16000_seed-90.txt
59f32632fdde7e12af7ce87a863313c6  valid_labels.txt
e43cdd4f3babdf14a496417843857e69  validpreacts_model_mnist_set-16000_seed-0.txt
212071b8a97c3b2a742e

In [0]:

![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd abstention
!git pull
!pip uninstall abstention
!pip install .
%cd ..

Cloning into 'abstention'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 294 (delta 45), reused 43 (delta 24), pack-reused 222
Receiving objects: 100% (294/294), 90.05 KiB | 4.29 MiB/s, done.
Resolving deltas: 100% (187/187), done.
/content/abstention
Already up to date.
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.3.0-cp36-none-any.whl size=45713 sha256=144f824dad42b0f61aa6ae37d67df93367ba62d296b17fd43bc3a86a6dba11bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-ej706xlp/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [0]:
![[ -e label_shift_experiments ]] || git clone https://github.com/blindauth/labelshiftexperiments
%cd /content/labelshiftexperiments
!git pull
!pip uninstall labelshiftexperiments
!pip install .
%cd ..

Cloning into 'labelshiftexperiments'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 23 (delta 8), reused 17 (delta 5), pack-reused 0
Unpacking objects: 100% (23/23), done.
/content/labelshiftexperiments
Already up to date.
Processing /content/labelshiftexperiments
  Created wheel for labelshiftexperiments: filename=labelshiftexperiments-0.1.0.0-cp36-none-any.whl size=6302 sha256=1ae741f649d6c5c9769a824752d31725ff090e46a7bac96023c46c73eff5d0cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-vuztfd_u/wheels/6d/56/a7/fc736645f87832f3f9af94ef6661744069f41cd66bab35ec1b
Successfully built labelshiftexperiments
/content


In [0]:
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
reload(abstention.figure_making_utils)
from abstention.calibration import (
    TempScaling, #CrossValidatedBCTS,
    ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter,
     BBSEImbalanceAdapter, ShiftWeightFromImbalanceAdapter)
import glob
import gzip
import numpy as np
from collections import defaultdict, OrderedDict

import labelshiftexperiments
reload(labelshiftexperiments)
reload(labelshiftexperiments.cifarandmnist)
from labelshiftexperiments import cifarandmnist
        
test_labels = cifarandmnist.read_preds(open("test_labels.txt"))
valid_labels = cifarandmnist.read_preds(open("valid_labels.txt"))

imbalanceadaptername_to_imbalanceadapter = {
    'em': EMImbalanceAdapter(),
    'bbse-hard': BBSEImbalanceAdapter(soft=False),
    'bbse-soft': BBSEImbalanceAdapter(soft=True)}

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4,5,6,7,8,9])),
    #('CVBCTS', CrossValidatedBCTS(
    #             verbose=True,
    #             num_crossvalidation_splits=5,
    #             betas_to_try=[0.0, 1e-7, 1e-6, 1e-5,
    #                           1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]))
])

adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'BCTS'),
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'BCTS'),
    #('em', 'CVBCTS'),
]

num_trials = 10
seeds = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
dirichlet_alphas_and_samplesize = [
    (0.1,2000), (1.0,2000),
    (0.1,4000), (1.0,4000),
    (0.1,8000), (1.0,8000)]
tweakone_alphas_and_samplesize = [
    (0.01,2000), (0.9,2000),
    (0.01,4000), (0.9,4000),
    (0.01,8000), (0.9,8000)]

print("Dirichlet shift")

(dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 dirichlet_alpha_to_samplesize_to_baselineacc,                        
 metric_to_samplesize_to_calibname_to_unshiftedvals) =\
  cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = dirichlet_alphas_and_samplesize,
    shifttype='dirichlet',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="validpreacts_model_mnist_set-16000_seed-",
    testglobprefix="testpreacts_model_mnist_set-16000_seed-",
    valid_labels=valid_labels,
    test_labels=test_labels)

print("Tweak one shift")

(tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,      
 tweakone_alpha_to_samplesize_to_baselineacc,                        
 _) = cifarandmnist.run_experiments(
    num_trials=num_trials,
    seeds=seeds,
    alphas_and_samplesize = tweakone_alphas_and_samplesize,
    shifttype='tweakone',
    calibname_to_calibfactory=calibname_to_calibfactory,
    imbalanceadaptername_to_imbalanceadapter=
      imbalanceadaptername_to_imbalanceadapter,
    adaptncalib_pairs=adaptncalib_pairs,
    validglobprefix="validpreacts_model_mnist_set-16000_seed-",
    testglobprefix="testpreacts_model_mnist_set-16000_seed-",
    valid_labels=valid_labels,
    test_labels=test_labels)



Dirichlet shift
Seed 0
Seed 10
Seed 20
Seed 30
Seed 40
Seed 50
Seed 60
Seed 70
Seed 80
Seed 90
Calibration stats
Metric ece
None 0.4815632366160694
TS 0.3613472630968465
BCTS 0.3430584138785365
Metric nll
None 0.046473676748765076
TS 0.045622743523019454
BCTS 0.04665527068305314
Metric jsdiv
None 0.003858078180335642
TS 0.004284419547940966
BCTS 0.004840155239455843
On alpha 0.1 sample size 2000
Metric delta_acc
bbse-hard:None 0.011080000000000017 +/- 0.0006212044345422452
bbse-soft:None 0.011110000000000017 +/- 0.0006174474990972233
bbse-soft:TS 0.01138000000000001 +/- 0.0006534338203489
bbse-soft:BCTS 0.011180000000000006 +/- 0.0006663453771252284
em:None 0.01184500000000001 +/- 0.000633161659348296
em:TS 0.012105000000000012 +/- 0.0006600541376633175
em:BCTS 0.011900000000000013 +/- 0.0006632868873643562
Metric jsdiv
bbse-hard:None 0.019821292832643098 +/- 0.001019496544735987
bbse-soft:None 0.01869964511243956 +/- 0.001042467365660921
bbse-soft:TS 0.016245258079594874 +/- 0.0009941

In [0]:
import json
import os
file_out = "label_shift_adaptation_results.json"
dict_to_write = {
    "dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "dirichlet_alpha_to_samplesize_to_baselineacc":
     dirichlet_alpha_to_samplesize_to_baselineacc,
    "metric_to_samplesize_to_calibname_to_unshiftedvals":
     metric_to_samplesize_to_calibname_to_unshiftedvals,
    "tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals":
     tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    "tweakone_alpha_to_samplesize_to_baselineacc":
     tweakone_alpha_to_samplesize_to_baselineacc
}
open(file_out, 'w').write(
    json.dumps(dict_to_write,
               sort_keys=True, indent=4, separators=(',', ': ')))
os.system("gzip -f "+file_out)

0

In [0]:
import gzip
import json
loaded_dicts = json.loads(gzip.open("label_shift_adaptation_results.json.gz").read())
dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
dirichlet_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['dirichlet_alpha_to_samplesize_to_baselineacc']
tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals']
tweakone_alpha_to_samplesize_to_baselineacc =\
    loaded_dicts['tweakone_alpha_to_samplesize_to_baselineacc']
metric_to_samplesize_to_calibname_to_unshiftedvals =\
    loaded_dicts['metric_to_samplesize_to_calibname_to_unshiftedvals']

In [0]:
from importlib import reload
import labelshiftexperiments
reload(labelshiftexperiments)
import labelshiftexperiments.maketable
reload (labelshiftexperiments.maketable)
from labelshiftexperiments.maketable import render_calibration_table

metricname_to_nicename = {'nll': 'nll', 'jsdiv': 'jsdiv', 'ece': 'ECE'}
calibname_to_nicename = {'None': "None", "TS": "TS", "BCTS": "BCTS"}
  
from scipy.stats import norm
N = len(seeds)*num_trials
#Using the normal approximation at N=100;
# variance from https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test
#Note that T = ((N+1)*N/2 - W)/2            
ustat_threshold = ((N*(N+1))/2 - norm.ppf(0.99)*np.sqrt(N*(N+1)*(2*N+1)/6.0))/2.0
  
print(render_calibration_table(
        metric_to_samplesize_to_calibname_to_unshiftedvals=
          metric_to_samplesize_to_calibname_to_unshiftedvals,
        #threshold of 8 comes from table https://www.oreilly.com/library/view/nonparametric-statistics-a/9781118840429/bapp02.xhtml
        #for one-tailed alpha=0.025 and n=10            
        ustat_threshold=ustat_threshold,
        metrics_in_table=['nll', 'jsdiv', 'ece'],
        samplesizes_in_table=['2000', '4000', '8000'],
        calibnames_in_table=['None', 'TS', 'BCTS'],
        metricname_to_nicename=metricname_to_nicename,
        calibname_to_nicename=calibname_to_nicename,
        caption="CIFAR10 Calibration metric differences", label="cifar10calibrationcomparison",
        applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c c c | c c c | c c c }
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{nll} & \multicolumn{3}{| c}{jsdiv} & \multicolumn{3}{| c}{ECE}\\
    \cline{2-10}
    & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000\\
    \hline
    None & 0.046; 1.32 & 0.046; 1.62 & 0.046; 1.84 & \textbf{0.004; 0.28} & 0.004; 0.51 & 0.004; 0.67 & 0.482; 1.56 & 0.482; 1.71 & 0.482; 1.75\\
    TS & \textbf{0.046; 0.5} & 0.045; 0.85 & 0.045; 0.92 & 0.004; 1.33 & 0.004; 1.63 & 0.004; 1.85 & 0.361; 0.79 & 0.336; 0.75 & 0.332; 0.7\\
    BCTS & 0.047; 1.18 & \textbf{0.045; 0.53} & \textbf{0.044; 0.24} & 0.005; 1.39 & 0.004; 0.86 & \textbf{0.003; 0.48} & 0.343; 0.65 & 0.32; 0.54 & 0.311; 0.55\\
  \end{tabular}}
  \caption{CIFAR10 Calibration metric differences}
  \label{tab:cifar10calibrationcomparison}
\end{table*}



In [0]:
from labelshiftexperiments.maketable import render_adaptation_table
    
methodgroups = OrderedDict([
  ('em', ['em:None', 'em:TS', 'em:BCTS']),
  ('bbse', ['bbse-hard:None', 'bbse-soft:None',
           'bbse-soft:TS', 'bbse-soft:BCTS'])])
samplesizes_in_table = ['2000', '4000', '8000']
adaptname_to_nicename = {'em': 'EM',
                           'bbse-soft': 'BBSE-soft',
                           'bbse-hard': 'BBSE-hard'}
calibname_to_nicename = {'None': 'None',
                           'TS': 'TS',
                           'BCTS': 'BCTS'}

dirichlet_alphas_in_table = ['0.1', '1.0']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: JS Divergence, dirichlet shift",
    label="cifar10jsdivdirichletshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=dirichlet_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=dirichlet_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: $\\Delta$\\%Accuracy, dirichlet shift",
    label="cifar10deltaaccdirichletshift",
    applyunderline=False))

tweakone_alphas_in_table = ['0.01', '0.9']

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=1.0,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='jsdiv',
    largerisbetter=False,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: JS Divergence, tweakone shift",
    label="cifar10jsdivtweakoneshift",
    applyunderline=False))

print(render_adaptation_table(
    alpha_to_samplesize_to_adaptncalib_to_metric_to_vals=tweakone_alpha_to_samplesize_to_adaptername_to_metric_to_vals,
    ustat_threshold=ustat_threshold,
    valmultiplier=100,
    adaptname_to_nicename=adaptname_to_nicename,
    calibname_to_nicename=calibname_to_nicename,
    methodgroups=methodgroups,
    metric='delta_acc',
    largerisbetter=True,
    alphas_in_table=tweakone_alphas_in_table,
    samplesizes_in_table=samplesizes_in_table,
    caption="CIFAR10 Metric: $\\Delta$\\%Accuracy, tweakone shift",
    label="cifar10deltaacctweakoneshift",
    applyunderline=False))


\begin{table*}
\adjustbox{max width=\textwidth}{
  \centering
  \begin{tabular}{ c | c | c c c | c c c}
    \multirow{2}{*}{\begin{tabular}{c}\textbf{Shift} \\ \textbf{Estimator} \end{tabular}} & \multirow{2}{*}{\begin{tabular}{c}\textbf{Calibration} \\ \textbf{Method} \end{tabular}} & \multicolumn{3}{| c}{$\alpha=0.1$} & \multicolumn{3}{| c}{$\alpha=1.0$}\\ 
    \cline{3-8}
    & & $n$=2000 & $n$=4000 & $n$=8000 & $n$=2000 & $n$=4000 & $n$=8000\\
    \hline
    \hline
    EM & None & 0.007; 1.18 & 0.007; 1.36 & 0.007; 1.41 & 0.006; 0.8 & 0.005; 0.96 & 0.005; 1.05\\
    EM & TS & 0.004; 0.85 & 0.005; 0.8 & 0.004; 0.87 & 0.006; 0.96 & 0.005; 1.1 & 0.005; 1.35\\
    EM & BCTS & 0.006; 0.97 & 0.005; 0.84 & 0.004; 0.72 & 0.007; 1.24 & 0.005; 0.94 & \textbf{0.004; 0.6}\\
    \hline
    \hline
    BBSE-hard & None & 0.02; 2.24 & 0.019; 2.31 & 0.016; 2.26 & 0.007; 1.42 & 0.006; 1.55 & 0.005; 1.62\\
    BBSE-soft & None & 0.019; 1.87 & 0.017; 2.04 & 0.015; 2.1 & 0.007; 1.3 & 0.005; 1.45 & 0.0